In [1]:
# import sys
# import requests

# !{sys.executable} -m pip install git+https://github.com/facebookresearch/segment-anything.git
    
# url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"
# r = requests.get(url)

# with open("sam_vit_h_4b8939.pth", "wb") as f:
#     f.write(r.content)

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from ultralytics import YOLO, settings
import shutil

print("CUDA is available:", torch.cuda.is_available())

CUDA is available: True


In [2]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

def show_image(image, ax='Off', figsize=(10, 10)):
    plt.figure(figsize=figsize)
    plt.imshow(image)
    plt.axis(ax)
    plt.show()

def show_image_with_boxes_and_masks(image_rgb, boxes, masks):
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    for mask in masks:
        show_mask(mask.cpu().numpy(), plt.gca(), random_color=True)
    for box in boxes:
        show_box(box.cpu().numpy(), plt.gca())
    plt.axis('off')
    plt.show()

In [3]:
MODEL_PATH_PLANT = '../models/plant_detection.pt'
settings.update({'runs_dir': rf'C:\Users\buyse\Workspace\NTNU\models\runs'})
model_plant = YOLO(MODEL_PATH_PLANT)

In [22]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator

sam_checkpoint = "../models/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)
mask_generator = SamAutomaticMaskGenerator(sam)

In [5]:
IMG_PATH_FIXED = '../images/classification/fixed'
IMG_PATH_RANDOM = '../images/classification/random'

if not os.path.exists(IMG_PATH_FIXED):
    print("Error, no fixed images found")
    exit(1)

if not os.path.exists(IMG_PATH_RANDOM):
    print("Error, no random images found")
    exit(1)

images = [os.path.join(IMG_PATH_FIXED, f) for f in os.listdir(IMG_PATH_FIXED) if f.endswith('.jpg')]
images += [os.path.join(IMG_PATH_RANDOM, f) for f in os.listdir(IMG_PATH_RANDOM) if f.endswith('.jpg')]

In [7]:
shutil.rmtree(settings['runs_dir'], ignore_errors=True)
results = model_plant(images, verbose=False)

### Single box output

In [8]:
output = []

for result in results:
    image_path = result.path
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    box = np.array(result.boxes.xyxy[0].tolist())

    predictor.set_image(image)

    masks, _, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=box,
    multimask_output=False,
    )

    output.append({'image_path': image_path, 'image': image, 'boxes': box, 'masks': masks})

In [ ]:
for i, o in enumerate(output):
    plt.figure(figsize=(10, 10))
    plt.imshow(o['image'])
    show_mask(o['masks'][0], plt.gca())
    show_box(o['boxes'], plt.gca())
    plt.axis('off')
    plt.show()

### Multi box output (not to good...)

In [9]:
output = []

for result in results:
    image_path = result.path
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = result.boxes.xyxy

    predictor.set_image(image)

    masks, _, _ = predictor.predict_torch(
    point_coords=None,
    point_labels=None,
    boxes=boxes,
    multimask_output=False,
    )

    output.append({'image_path': image_path, 'image': image, 'boxes': boxes, 'masks': masks})

In [ ]:
for i, res in enumerate(output):
    show_image_with_boxes_and_masks(res['image'], res['boxes'], res['masks'])
    print(res['image_path'])

### Crop out multiple bounding boxes first, then predict with SAM

In [25]:
image = cv2.imread('../images/classification/fixed/1701249276.jpg')

shutil.rmtree(settings['runs_dir'], ignore_errors=True)
results = model_plant(image, verbose=False, save_crop=True)

PATH = settings['runs_dir'] + '/detect/predict/crops/plant'
crops = [os.path.join(PATH, f) for f in os.listdir(PATH) if f.endswith('.jpg')]

Results saved to C:\Users\buyse\Workspace\NTNU\models\runs\detect\predict
1 label saved to C:\Users\buyse\Workspace\NTNU\models\runs\detect\predict\labels


In [ ]:
for img in crops:

    img = cv2.imread(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    middle_point = np.array([[img.shape[1] // 2, img.shape[0] // 2]])

    predictor.set_image(img)
    masks, _, _ = predictor.predict(
    point_coords=middle_point,
    point_labels=np.array([1]),
    multimask_output=False,
    )

    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    show_mask(masks, plt.gca())
    plt.axis('off')
    plt.show() 